In [14]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
from typing import List, Dict
import matplotlib.pyplot as plt
import networkx as nx
%matplotlib qt 

In [2]:
load_dotenv('../featuring_network.env')
client = os.getenv('SPOTIFY_CLIENT', 'client')
secret = os.getenv('SPOTIFY_SECRET', 'secret')

In [3]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client, client_secret=secret))

In [4]:
def add_song_edge(artist1, artist2, song_data, song_id, graph):
    """
    Args:
        artist1 - 
        artist2 - 
        song_data - 
        song_id - 
        graph - 
    
    Returns:
    
    """
    graph.add_edge(artist1, artist2,
               name=song_data['name'],
               album_name=song_data['album_name'],
               song_url=song_data['song_url']['spotify'],
               song_id=song_id )


def add_feature_edges(artist_name: str, track_list: Dict, graph: nx.Graph) -> None:
    """
    Args:
        artist_name - 
        track_list - 
        graph - 
    
    Returns: 
    
    """
    for track in track_list:
        for collaborator in track_list[track]['features']:
            if collaborator not in graph:
                add_artist_node(collaborator, graph)
            add_song_edge(artist_name, collaborator, track_list[track],track,  graph)



def add_artist_node(artist_name: str, graph: nx.Graph) -> None:
    """
    Args:
        artist_name - 
        graph - 
    Returns:
    
    """
    artist = get_artist_id(artist_name)    
    if artist is None:
        print('Artist ', artist_name, ' not found!')
        return 
    album_ids = get_artist_album_ids(artist['id'])
    if album_ids is None:
        album_ids = []
    graph.add_node(
        artist['name'], 
        aid=artist['id'],
        albums=set(album_ids),
        genres=artist['genres'],
        followers=artist['followers']['total'],
        href=artist['href'],
        uri=artist['uri'],
        popularity=artist['popularity']
    )


def get_artist_id(name: str) -> str:
    """
    Args:
        name -

    """
    results = sp.search(q='artist:' + name, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        return items[0]
    else:
        return None
    
def get_artist_album_ids(artist_id: str) -> List:
    """
    Args:
        artist_id -
    
    Returns:
    
    """
    album_names = set()
    albums = set()
    results = sp.artist_albums(artist_id, album_type='album')
    for album in results['items']:
        if album['name'] not in album_names:
            albums.add(album['id'])
            album_names.add(album['name'])
    return list(albums)
    
def get_album_track_ids(album_ids: List, artist_id: str) -> Dict:
    """ uses spotipy to get track information for albums in list
    Args:
        album_ids - 
        artist_id - 
        
    Returns:
    
    """
    songs_checker = set()
    songs_w_features = {}
    for album in album_ids:
        track_ids = sp.album_tracks(album)['items']
        for track in track_ids:
            features = [x['name'] for x in track['artists'] if x['uri'] != artist_id]
            # exclude songs w. no features
            if len(features)>1 and track['name'] not in songs_checker:
                songs_w_features[track['id']] = {}
                songs_w_features[track['id']]['name'] = track['name']
                songs_w_features[track['id']]['features'] = features
                songs_w_features[track['id']]['album_name'] = album
                songs_w_features[track['id']]['song_url'] = track['external_urls']
                songs_checker.add(track['name'])

    return songs_w_features


In [15]:

artist = 'Kanye West'
graph = nx.Graph()
add_artist_node(artist, graph)
song_ids = get_album_track_ids(graph.nodes[artist]['albums'], graph.nodes[artist]['aid'])
add_feature_edges(artist, song_ids, graph)


print(graph.nodes)
print(graph.edges)



Artist  D'banj  not found!
['Kanye West', 'T-Pain', 'Lil Wayne', 'Mos Def', 'Dwele', 'DJ Premier', 'Chris Martin', 'Albe Back', 'Sunday Service Choir', 'Ty Dolla $ign', 'Ant Clemons', 'Fred Hammond', 'Clipse', 'Kenny G', 'JAY-Z', 'Frank Ocean', 'The-Dream', 'Beyoncé', 'Otis Redding', 'Mr Hudson', 'Curtis Mayfield', 'PARTYNEXTDOOR', 'KIDS SEE GHOSTS', 'Pusha T', 'Louis Prima', 'Yasiin Bey', 'André 3000', 'R. Kelly', 'Teyana Taylor', 'Big Sean', '2 Chainz', 'Ghostface Killah', 'Raekwon', 'Common', 'CyHi The Prynce', 'Kid Cudi', "D'banj", 'DJ Khaled', 'Bob Marley & The Wailers', 'Ma$e', 'Cocaine 80s', 'John Legend', 'Travis Scott', 'Malik Yusef', 'Marsha Ambrosius', 'Chief Keef', 'Jadakiss', 'Rick Ross', 'Nicki Minaj', 'Bon Iver', 'Swizz Beatz', 'RZA']
[('Kanye West', 'Kanye West'), ('Kanye West', 'T-Pain'), ('Kanye West', 'Lil Wayne'), ('Kanye West', 'Mos Def'), ('Kanye West', 'Dwele'), ('Kanye West', 'DJ Premier'), ('Kanye West', 'Chris Martin'), ('Kanye West', 'Albe Back'), ('Kanye Wes

In [16]:


subax1 = plt.subplot(121)
nx.draw(graph, with_labels=True, font_weight='bold')
plt.show()